In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 2 11:36:10 2022

@author: Eric Larmanou
Data from Campbell dataloggers visualization
"""

import datetime
import ipywidgets as wg

from bokeh.models.widgets.inputs import AutocompleteInput
from IPython.display import display
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from library_Campbell import SiteSet

dark_template = pio.templates["plotly_dark"]

#import the list of sites and dataloggers
FileLoggers = r'C:\MyDoc\prog\jupyter\CampbellViz\loggers.csv'
Set = SiteSet(FileLoggers)

Sites = pd.unique(Set.Loggers.Site)
Site = Sites[1]
Ids = Set.Loggers.loc[Set.Loggers.Site == Site].index
Id = Ids.to_list()[-1]
if not Set.Tables[Id].Header.columns.empty:
    Variables = Set.Tables[Id].Header.columns[1:]
    Variable = Variables[0]
else:
    Variables = []
    Variable = None

#create widgets
DatePickerStart = wg.DatePicker(description='Start date')
DatePickerEnd = wg.DatePicker(description='End date')
DropdownSite = wg.Dropdown(description = 'Site:', options = Sites, selected_label=Site, value=Site)
DropdownId = wg.Dropdown(description = 'Table:', options = Ids, selected_label=Id, value=Id)
ButtonLoad = wg.Button(description='Load data')
DropdownVariable = wg.Dropdown(description = 'Variable:', options = Variables)
ButtonAddTrace = wg.Button(description='Add trace')
ButtonResetPlot = wg.Button(description='Reset plot')
ValidReady = wg.Valid(value=True, description='Ready')
LabelData = wg.Label(value='Empty', layout={'border': '1px solid grey'})
FigureTime = go.FigureWidget(layout={'template':dark_template, 'title':'time serie'})
FigureHist = go.FigureWidget(layout={'template':dark_template, 'title':'histogram'})

global_vbox = wg.VBox()
global_vbox.children = [wg.HBox([DatePickerStart,DatePickerEnd,DropdownSite,DropdownId,ButtonLoad]), wg.HBox([DropdownVariable,ButtonAddTrace,ButtonResetPlot, ValidReady, LabelData]), FigureTime, FigureHist]
display(global_vbox)

def SiteChange(x):
    global DropdownSite, DropdownId
    Site = DropdownSite.value
    Loggers = Set.Loggers.loc[Set.Loggers.Site == Site].index
    #DropdownLogger.index = None #This line is very important for setting the values for widgets other than widget with observe method
    #DropdownLogger.index = 0
    DropdownId.options = Loggers
    
def IdChange(x):
    global Id
    Id = DropdownId.value
    UpdateVariables()
    
def UpdateVariables():
    #update the list of variables shown in the selection list
    global Id
    Variables = Set.Tables[Id].Header.columns[1:].to_list()
    if (not Variables) and hasattr(Set.Tables[Id], 'DF_data'):
        Variables = Set.Tables[Id].DF_data.columns
    DropdownVariable.options = Variables
    if hasattr(Set.Tables[Id], 'DF_data'):
        if (Set.Tables[Id].DateStart is None) and (Set.Tables[Id].DateEnd is None):
            LabelData.value = 'All'
        elif (Set.Tables[Id].DateStart is None):
            LabelData.value = 'Until ' + str(Set.Tables[Id].DateEnd)
        elif (Set.Tables[Id].DateEnd is None):
            LabelData.value = 'From ' + str(Set.Tables[Id].DateStart)
        else:
            LabelData.value = 'From ' + str(Set.Tables[Id].DateStart) + ' to ' + str(Set.Tables[Id].DateEnd)
    else:
        LabelData.value = 'Empty'

def Load(x):
    global Id
    ValidReady.value = False
    DateStart = DatePickerStart.value
    DateEnd =  DatePickerEnd.value
    if not DatePickerStart.value is None: # if not None, convert date in to datetime
        DateStart = datetime.datetime.combine(DateStart, datetime.datetime.min.time())
    if not DatePickerEnd.value is None:
        DateEnd =  datetime.datetime.combine(DateEnd, datetime.datetime.min.time())
    #load data
    Set.LoadData([Id,], DateStart, DateEnd)
    UpdateVariables()
    ValidReady.value = True
    
def AddTrace(x):
    global Id
    #load data if not done yet
    if not hasattr(Set.Tables[Id], 'DF_data'):
        Load(x)
    ValidReady.value = False
    Variable = DropdownVariable.value
    print('Plotting ' + Variable)
    FigureTime.add_scatter(x=Set.Tables[Id].DF_data.index, y=Set.Tables[Id].DF_data[Variable], name=Id+'/'+Variable, connectgaps=False)
    FigureHist.add_histogram(y=Set.Tables[Id].DF_data[Variable], name=Id+'/'+Variable)
    ValidReady.value = True
    
def ResetPlot(x):
    FigureTime.data = []
    FigureHist.data = []

DropdownSite.observe(SiteChange)
DropdownId.observe(IdChange)
ButtonLoad.on_click(Load)
ButtonAddTrace.on_click(AddTrace)
ButtonResetPlot.on_click(ResetPlot)

Scan of the folder "C:\MyDoc\data\backupZac\ICOS\DATA\GL-ZaH\met\GL_Zaf*.dat"
